**Why TileDB?**

What got me really interested in TileDB is that it operates as a severless database. You can store it as a file structure in AWS S3, and more importantly, you can seamlessly _update_ the database as it is stored in S3 also.

This may sound strange, since S3 is an append-only system. You may ask, wouldn't this mean re-uploading GBs worth of data each time? Turns out TileDB get around this by exploiting AWS S3's _multi-part upload_ API, which allows writing in chunks of 5MB. So this lets them efficiently do random-access writing to an append-only data structure.

https://docs.tiledb.com/main/how-to/backends/s3#performance

Also, the TileDB data structure isn't actually a single file - it's a directory, almost akin to a git repo in a way, with commits for handling versioning. It's cleverly designed to get all the qualities you might expect from a database:

* [atomicity](https://docs.tiledb.com/main/background/internal-mechanics/atomicity)
* [consistency](https://docs.tiledb.com/main/background/internal-mechanics/consistency) (eventual-consistency, or [S3's read-after-write consistency](https://aws.amazon.com/blogs/aws/amazon-s3-update-strong-read-after-write-consistency/)
* [concurrency](https://docs.tiledb.com/main/background/internal-mechanics/concurrency) (no need for locking, as writes are done as separate fragments and do not need to be synchronised, fragments are resolved by periodic "vacuuming")



**TileDB for Geometries?**

I suggest taking a look at these articles for how TileDB works with geometries

https://tiledb.com/blog/why-tiledb-for-geometries

https://tiledb.com/blog/tiledb-101-geometries

TLDR:

* it takes the centroid of geometries and indexes them as points for fast searching, with the geometry as an attribute
* spatial indexing is built-in this way, and gets updated generically each time it is written to
* points are stored following the GDAL standard `_X` and `_Y` convention, so that means GDAL can easily search through them

# reading through REMA STAC

In [1]:
import pystac

In [2]:
import pystac
# from pystac_client import Client
from shapely.geometry import box

# Define the URL of the STAC Collection
collection_url = "https://pgc-opendata-dems.s3.us-west-2.amazonaws.com/rema/mosaics/v2.0/32m.json"

# Open the collection
collection = pystac.Collection.from_file(collection_url)

In [4]:
tile = pystac.Catalog.from_file(collection.links[3].href)

In [6]:
feature = pystac.Item.from_file(tile.links[3].href)

In [8]:
feature.geometry

{'type': 'MultiPolygon',
 'coordinates': [[[[165.96537437139298, -67.51394127561791, 0.0],
    [163.7370670246123, -67.28182311677956, 0.0],
    [164.0530227378955, -66.85501128483631, 0.0],
    [166.24176511403135, -67.08236170623843, 0.0],
    [165.96537437139298, -67.51394127561791, 0.0]]]]}

# creating tiledb dataset

In [12]:
child_links = collection.get_child_links()

In [13]:
child_links[0].href

'https://pgc-opendata-dems.s3.us-west-2.amazonaws.com/rema/mosaics/v2.0/32m/06_37.json'

In [14]:
list(pystac.Catalog.from_file(child_links[0].href).get_children())[0]

<Item id=06_37_32m_v2.0>

In [15]:
from pathlib import Path
import shapely
from tqdm import tqdm

def process_tile(child):
    feature = list(pystac.Catalog.from_file(child.href).get_children())[0]
    feature_id = feature.id
    dem = str(Path(feature.get_self_href()).parent.joinpath(feature.assets['dem'].href))
    hillshade = str(Path(feature.get_self_href()).parent.joinpath(feature.assets['hillshade'].href))
    geometry = shapely.geometry.shape(feature.geometry)

    return {
        'id': feature_id,
        'dem': dem,
        'hillshade': hillshade,
        'geometry': geometry
    }

In [16]:
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(max_workers=100) as executor:
    futures = executor.map(process_tile, child_links)
    result = list(tqdm(futures, total=len(child_links)))

100%|██████████████████████████████████████| 1523/1523 [00:50<00:00, 30.10it/s]


In [18]:
import pandas as pd

In [19]:
df = pd.DataFrame.from_dict(result)

In [20]:
import geopandas as gpd

In [21]:
gdf = gpd.GeoDataFrame(df[['id', 'dem', 'hillshade']], geometry=df['geometry'])

In [22]:
gdf.to_file('rema.shp', driver='ESRI Shapefile')

In [26]:
!ogr2ogr -f "TileDB" rema-32m.tiledb rema.shp -lco BOUNDS="-180.0,-90.0,180.0,90.0"

In [28]:
import tiledb

In [29]:
with tiledb.Array('rema-32m.tiledb') as A:
    print(A.schema)

ArraySchema(
  domain=Domain(*[
    Dim(name='_X', domain=(-180.0, 180.0), tile=18.0, dtype='float64'),
    Dim(name='_Y', domain=(-90.0, 90.0), tile=18.0, dtype='float64'),
    Dim(name='_Z', domain=(-10000.0, 10000.0), tile=10000.0, dtype='float64'),
  ]),
  attrs=[
    Attr(name='FID', dtype='int64', var=False, nullable=False, enum_label=None),
    Attr(name='wkb_geometry', dtype='blob', var=True, nullable=False, enum_label=None),
    Attr(name='id', dtype='<U0', var=True, nullable=True, enum_label=None),
    Attr(name='dem', dtype='<U0', var=True, nullable=True, enum_label=None),
    Attr(name='hillshade', dtype='<U0', var=True, nullable=True, enum_label=None),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=True,
  allows_duplicates=True,
)



In [30]:
with tiledb.open('rema-32m.tiledb', 'r') as array:
    data = array[:]
    print(data)

OrderedDict([('FID', array([164, 202, 241, ...,  10,  22,   9])), ('wkb_geometry', array([array([   1,  -21,    3,    0,    0,    1,    0,    0,    0,    5,    0,
                 0,    0,  -11,   20,  -75,    7,   10,   81,  101,  -64,   40,
                50, -120,  -48,  -95,  -75,   83,  -64,    0,    0,    0,    0,
                 0,    0,    0,    0,   78,   74,  -34,  110,  -78,  -25,  101,
               -64,  -37, -104,  -35,  125,  -31,  -68,   83,  -64,    0,    0,
                 0,    0,    0,    0,    0,    0, -118,  -91,   43,  -23,   98,
               -13,  101,  -64,  -73,   -8,  -93,  -78,  -98, -126,   83,  -64,
                 0,    0,    0,    0,    0,    0,    0,    0,  104, -114,  110,
               -54,    2,  104,  101,  -64,  -44,  114,  111,  127,  -18,  123,
                83,  -64,    0,    0,    0,    0,    0,    0,    0,    0,  -11,
                20,  -75,    7,   10,   81,  101,  -64,   40,   50, -120,  -48,
               -95,  -75,   83,  -64,

# reading from S3

manually synced it to a *public* S3 bucket

$ aws s3 sync ./rema-32m.tiledb s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb

to use TileDB with S3 datasets, you need to make sure TileDB is compiled with the `--enable-s3` flag

for conda, as used here, this seems to be the case, but your mileage may vary

the story gets even more complicated when using TileDB together with GDAL, which will need at least GDAL 3.7 which is when TileDB 2.7 GDAL driver was added

In [1]:
import tiledb
config = tiledb.Config()
config["vfs.s3.region"] = "ap-southeast-2"

# since it's a public S3 bucket, I shouldn't need to set AWS credentials
# vfs.s3.no_sign_request option works, but isn't documented in https://docs.tiledb.com/main/how-to/backends/s3
config["vfs.s3.no_sign_request"] = "true"
ctx = tiledb.Ctx(config)

it's actually quite slow to read from S3 for the first time

In [2]:
%%time
with tiledb.Array('s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb', ctx=ctx) as A:
    print(A.schema)

ArraySchema(
  domain=Domain(*[
    Dim(name='_X', domain=(-180.0, 180.0), tile=18.0, dtype='float64'),
    Dim(name='_Y', domain=(-90.0, 90.0), tile=18.0, dtype='float64'),
    Dim(name='_Z', domain=(-10000.0, 10000.0), tile=10000.0, dtype='float64'),
  ]),
  attrs=[
    Attr(name='FID', dtype='int64', var=False, nullable=False, enum_label=None),
    Attr(name='wkb_geometry', dtype='blob', var=True, nullable=False, enum_label=None),
    Attr(name='id', dtype='<U0', var=True, nullable=True, enum_label=None),
    Attr(name='dem', dtype='<U0', var=True, nullable=True, enum_label=None),
    Attr(name='hillshade', dtype='<U0', var=True, nullable=True, enum_label=None),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=True,
  allows_duplicates=True,
)

CPU times: user 231 ms, sys: 24.7 ms, total: 255 ms
Wall time: 6.28 s


it seems to get faster the second time it's read - implying some caching is done:

In [5]:
%%time
with tiledb.Array('s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb', ctx=ctx) as A:
    print(A.schema)

ArraySchema(
  domain=Domain(*[
    Dim(name='_X', domain=(-180.0, 180.0), tile=18.0, dtype='float64'),
    Dim(name='_Y', domain=(-90.0, 90.0), tile=18.0, dtype='float64'),
    Dim(name='_Z', domain=(-10000.0, 10000.0), tile=10000.0, dtype='float64'),
  ]),
  attrs=[
    Attr(name='FID', dtype='int64', var=False, nullable=False, enum_label=None),
    Attr(name='wkb_geometry', dtype='blob', var=True, nullable=False, enum_label=None),
    Attr(name='id', dtype='<U0', var=True, nullable=True, enum_label=None),
    Attr(name='dem', dtype='<U0', var=True, nullable=True, enum_label=None),
    Attr(name='hillshade', dtype='<U0', var=True, nullable=True, enum_label=None),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=True,
  allows_duplicates=True,
)

CPU times: user 161 ms, sys: 13.1 ms, total: 174 ms
Wall time: 132 ms


some forum posts seemed to be talking about large fragement counts leading to slow load times,
but that doesn't seem to be the case here (I've only got one fragment, because I've only just made it):

In [12]:
fragments_info = tiledb.array_fragments("rema-32m.tiledb")

# Number of fragments
print(len(fragments_info))

# URI of given fragment, with 0 <= idx < numfrag
print(fragments_info.uri[0])

# Timestamp range of given fragment, with 0 <= idx < numfrag
print(fragments_info.timestamp_range[0])

1
file:///home/jonathan/deant/tiledb/rema-32m.tiledb/__fragments/__1721972021646_1721972021646_700ba35eff0f0127ef5b652dee7e8ae0_21
(1721972021646, 1721972021646)


In [5]:
!time aws s3 ls s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb --no-sign-request

                           PRE rema-32m.tiledb/

real	0m4.782s
user	0m0.594s
sys	0m0.106s


This forum post discovers a nugget of the underlying S3 calls slowing things down: https://forum.tiledb.com/t/s3-first-access-very-slow-with-3d-tiled-dense-array/416/16

and can sped up significantly to sub-second times by setting `export AWS_EC2_METADATA_DISABLED=true`

(tested by reloading the notebook kernel from scratch)

In [2]:
import os
os.environ["AWS_EC2_METADATA_DISABLED"] = "true"

In [3]:
%%time
with tiledb.Array('s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb', ctx=ctx) as A:
    print(A.schema)

ArraySchema(
  domain=Domain(*[
    Dim(name='_X', domain=(-180.0, 180.0), tile=18.0, dtype='float64'),
    Dim(name='_Y', domain=(-90.0, 90.0), tile=18.0, dtype='float64'),
    Dim(name='_Z', domain=(-10000.0, 10000.0), tile=10000.0, dtype='float64'),
  ]),
  attrs=[
    Attr(name='FID', dtype='int64', var=False, nullable=False, enum_label=None),
    Attr(name='wkb_geometry', dtype='blob', var=True, nullable=False, enum_label=None),
    Attr(name='id', dtype='<U0', var=True, nullable=True, enum_label=None),
    Attr(name='dem', dtype='<U0', var=True, nullable=True, enum_label=None),
    Attr(name='hillshade', dtype='<U0', var=True, nullable=True, enum_label=None),
  ],
  cell_order='row-major',
  tile_order='row-major',
  capacity=10000,
  sparse=True,
  allows_duplicates=True,
)

CPU times: user 259 ms, sys: 88.9 ms, total: 348 ms
Wall time: 246 ms


# loading with gdal

GDAL has this thing called open options (OO), which the TileDB driver can be passed configuration

https://gdal.org/drivers/vector/tiledb.html

we can take our `tiledb.config()` (including the `vfs.s3.no_sign_request`) and save it to a file

In [4]:
config.save('tiledb_config.txt')

In [5]:
! ogrinfo -OO TILEDB_CONFIG=tiledb_config.txt -al s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb -spat 96.8173512092876 -65.5945721388298 97.8173512092876 -64.5945721388298

INFO: Open of `s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb'
      using driver `TileDB' successful.

Layer name: rema-32m
Geometry: 3D Polygon
Feature Count: 2
Extent: (-180.000000, -89.080536) - (180.000000, -53.544475)
Layer SRS WKT:
(unknown)
FID Column = FID
Geometry Column = wkb_geometry
id: String (0.0)
dem: String (0.0)
hillshade: String (0.0)
OGRFeature(rema-32m):575
  id (String) = 27_58_32m_v2.0
  dem (String) = https:/pgc-opendata-dems.s3.us-west-2.amazonaws.com/rema/mosaics/v2.0/32m/27_58/27_58_32m_v2.0_dem.tif
  hillshade (String) = https:/pgc-opendata-dems.s3.us-west-2.amazonaws.com/rema/mosaics/v2.0/32m/27_58/27_58_32m_v2.0_browse.tif
  POLYGON Z ((96.3384029298158 -65.3693078332198 0,96.2236728053973 -64.9307335887834 0,98.2776140541491 -64.8196335354118 0,98.4292578442117 -65.2559935220507 0,96.3384029298158 -65.3693078332198 0))

OGRFeature(rema-32m):574
  id (String) = 27_57_32m_v2.0
  dem (String) = https:/pgc-opendata-dems.s3.us-west-2.amazonaws.co

In [6]:
! gdalinfo -OO TILEDB_CONFIG=tiledb_config.txt s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb

Driver: TileDB/TileDB
Files: none associated
Size is 512, 512
Corner Coordinates:
Upper Left  (    0.0,    0.0)
Lower Left  (    0.0,  512.0)
Upper Right (  512.0,    0.0)
Lower Right (  512.0,  512.0)
Center      (  256.0,  256.0)


for the python osgeo gdal wrapper, I had success using `gdal.OpenEx()`, as it offers the `open_options` parameter

In [8]:
from osgeo import gdal

In [11]:
dataset = gdal.OpenEx("s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb",
                      open_options=["TILEDB_CONFIG=tiledb_config.txt"],
                      allowed_drivers=["TileDB"])

xmin, ymin, xmax, ymax = 96.8173512092876, -65.5945721388298, 97.8173512092876, -64.5945721388298

layer = dataset.GetLayer()
layer.SetSpatialFilterRect(xmin, ymin, xmax, ymax)
layer.ResetReading()
for feature in layer:
    print(feature.ExportToJson())

{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[96.3384029298158, -65.36930783321982, 0.0], [96.22367280539731, -64.93073358878341, 0.0], [98.27761405414907, -64.81963353541184, 0.0], [98.42925784421169, -65.25599352205069, 0.0], [96.3384029298158, -65.36930783321982, 0.0]]]}, "properties": {"id": "27_58_32m_v2.0", "dem": "https:/pgc-opendata-dems.s3.us-west-2.amazonaws.com/rema/mosaics/v2.0/32m/27_58/27_58_32m_v2.0_dem.tif", "hillshade": "https:/pgc-opendata-dems.s3.us-west-2.amazonaws.com/rema/mosaics/v2.0/32m/27_58/27_58_32m_v2.0_browse.tif"}, "id": 575}
{"type": "Feature", "geometry": {"type": "Polygon", "coordinates": [[[96.58009773221518, -66.2451758007948, 0.0], [96.33795588285456, -65.36762844612387, 0.0], [98.42866706212173, -65.25432275551286, 0.0], [98.74854689956759, -66.12720895554077, 0.0], [96.58009773221518, -66.2451758007948, 0.0]]]}, "properties": {"id": "27_57_32m_v2.0", "dem": "https:/pgc-opendata-dems.s3.us-west-2.amazonaws.com/rema/mosaics/v2

In [49]:
# geopandas passes kwargs through to pyogrio, which passes it to GDAL
gdf = gpd.read_file("s3://deant-data-public-dev/experimental/rema/rema-32m.tiledb",
              engine="pyogrio",
              use_arrow=True,
              bbox=(xmin, ymin, xmax, ymax),
              TILEDB_CONFIG="tiledb_config.txt"
             )
gdf

,id,dem,hillshade,geometry
0,27_58_32m_v2.0,https:/pgc-opendata-dems.s3.us-west-2.amazonaw...,https:/pgc-opendata-dems.s3.us-west-2.amazonaw...,"POLYGON Z ((96.3384 -65.36931 0, 96.22367 -64...."
1,27_57_32m_v2.0,https:/pgc-opendata-dems.s3.us-west-2.amazonaw...,https:/pgc-opendata-dems.s3.us-west-2.amazonaw...,"POLYGON Z ((96.5801 -66.24518 0, 96.33796 -65...."
